# Investing with Historical Yahoo Finance Data

In [25]:
import pandas as pd
import os
import time
from datetime import datetime
import bs4 as b4
from bs4 import BeautifulSoup
import re
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
style.use("dark_background")

matplotlib.rcParams['figure.figsize']
plt.rcParams["figure.figsize"] = [12,9]
matplotlib.rcParams['figure.figsize']


[12.0, 9.0]

# Gathering + Organizing Data

In [26]:
#Gathered Data from historical yahoo finance data for stock prices
#Gathered SP500 indices
path  = "intraQuarter"
def Key_Stats(gather=["Total Debt/Equity",
                      'Trailing P/E',
                      'Price/Sales',
                      'Price/Book',
                      'Profit Margin',
                      'Operating Margin',
                      'Return on Assets',
                      'Return on Equity',
                      'Revenue Per Share',
                      'Market Cap',
                        'Enterprise Value',
                        'Forward P/E',
                        'PEG Ratio',
                        'Enterprise Value/Revenue',
                        'Enterprise Value/EBITDA',
                        'Revenue',
                        'Gross Profit',
                        'EBITDA',
                        'Net Income Avl to Common ',
                        'Diluted EPS',
                        'Earnings Growth',
                        'Revenue Growth',
                        'Total Cash',
                        'Total Cash Per Share',
                        'Total Debt',
                        'Current Ratio',
                        'Book Value Per Share',
                        'Cash Flow',
                        'Beta',
                        'Held by Insiders',
                        'Held by Institutions',
                        'Shares Short (as of',
                        'Short Ratio',
                        'Short % of Float',
                        'Shares Short (prior ']):
    
    statspath = path +'/_KeyStats'
    stock_list = [x[0] for x in os.walk(statspath)] #Prints all directories from top down
    df = pd.DataFrame(columns = ['Date',
                                 'Unix',
                                 'Ticker',
                                 'Price',
                                 'stock_p_change',
                                 'SP500',
                                 'sp500_p_change',
                                 'Difference',
                                 ##############
                                 'DE Ratio',
                                 'Trailing P/E',
                                 'Price/Sales',
                                 'Price/Book',
                                 'Profit Margin',
                                 'Operating Margin',
                                 'Return on Assets',
                                 'Return on Equity',
                                 'Revenue Per Share',
                                 'Market Cap',
                                 'Enterprise Value',
                                 'Forward P/E',
                                 'PEG Ratio',
                                 'Enterprise Value/Revenue',
                                 'Enterprise Value/EBITDA',
                                 'Revenue',
                                 'Gross Profit',
                                 'EBITDA',
                                 'Net Income Avl to Common ',
                                 'Diluted EPS',
                                 'Earnings Growth',
                                 'Revenue Growth',
                                 'Total Cash',
                                 'Total Cash Per Share',
                                 'Total Debt',
                                 'Current Ratio',
                                 'Book Value Per Share',
                                 'Cash Flow',
                                 'Beta',
                                 'Held by Insiders',
                                 'Held by Institutions',
                                 'Shares Short (as of',
                                 'Short Ratio',
                                 'Short % of Float',
                                 'Shares Short (prior ',                                
                                 ##############
                                 'Status'])
    
    #Load in S&P 500 CSV

    sp500_df = pd.DataFrame.from_csv("sp500.csv")

    #iterate through each directory
    ticker_list = []
    for each_dir in stock_list[1:]:
        each_file = os.listdir(each_dir)
        try:
            starting_stock_value = False
            starting_sp500_value = False
            ticker = each_dir.split("/")[2]
            ticker_list.append(ticker)
            if len(each_file) > 0:
                for file in each_file:
                    date_stamp = datetime.strptime(file, '%Y%m%d%H%M%S.html')
                    unix_time = time.mktime(date_stamp.timetuple())
                    full_file_path = each_dir + "/" + file
                    source = open(full_file_path, 'r').read()
                    soup = BeautifulSoup(source, 'html.parser')
                    try:
                        value_list = []
                        for each_data in gather:
                            try: 
                                pattern_first = '.*?(\d{1,8}\.\d{1,8}M?B?K?|N/A)%?</td>'
                                finding_values= re.compile(re.escape(each_data)+ pattern_first)
                                value = re.findall(finding_values, source)
                                value = value[0]
#                                 regex_first = re.escape(each_data)
#                                 print(regex_first)
#                                 value = re.findall(regex_first, source)
#                                 print(value)
                                
                                if "B" in value:
                                    value = float(value.replace("B","")) * 1000000000
                                elif "M" in value:
                                    value = float(value.replace("M","")) * 1000000
                                elif "K" in value:
                                    value = float(value.replace("K","")) * 1000
                                value_list.append(value)

                            except Exception as e:
                                print(str(e))
                                value = "N/A"
                                value_list.append(value)
                        try:
                            sp500_date = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d')
#                            
#                             row = sp500_df[(date_stamp == sp500_date)]
#                             sp500_value = float(row['Adj Close'])
#                             print(float(sp500_df[sp500_df.index == sp500_date]['Adj Close']))
                            sp500_value = float(sp500_df[sp500_df.index == sp500_date]['Adj Close'])

                        except Exception as e: 
#                             print(e, file, '-----------------------')
                            sp500_date = datetime.fromtimestamp(unix_time - 259200).strftime('%Y-%m-%d')
#                             row = sp500_df[(sp500_df.index ==sp500_date)]
#                             sp500_value = float(row['Adj Close'])
                            sp500_value = float(sp500_df[sp500_df.index == sp500_date]['Adj Close'])

                        try:
                            stock_price = float(source.split('</small><big><b>')[1].split('</b></big>')[0])
                        except Exception as e:
                            try:
                                regex = '[0-9]+\.[0-9]+'
                                pattern = re.compile(regex)
                                stock_price = re.findall(pattern, str(stock_price))
    #                         stock_price = re.search(r '(\d{1,8}\.\d{1,8})', stock_price)
                                stock_price = float(stock_price[0])
                            except Exception as e:
                                print(str(e))
                                pass
    
                        if starting_stock_value == False:
                            starting_stock_value = stock_price
                        if starting_sp500_value == False:
                            starting_sp500_value = sp500_value
                        stock_p_change = ((stock_price - starting_stock_value) / starting_stock_value)*100
                        sp500_p_change = ((sp500_value - starting_sp500_value) / starting_sp500_value)*100
                        difference = stock_p_change - sp500_p_change
                        if difference > 0:
                            status = 'outperformed market'
                        else:
                            status = "underperformed"
                            
                        if value_list.count("N/A") > 0:
                            pass
                        else:
                        
                           df = df.append({'Date':date_stamp,
                                            'Unix':unix_time,
                                            'Ticker':ticker,
                                            
                                            'Price':stock_price,
                                            'stock_p_change':stock_p_change,
                                            'SP500':sp500_value,
                                            'sp500_p_change':sp500_p_change,
                                            'Difference':difference,
                                            'DE Ratio':value_list[0],
                                            #'Market Cap':value_list[1],
                                            'Trailing P/E':value_list[1],
                                            'Price/Sales':value_list[2],
                                            'Price/Book':value_list[3],
                                            'Profit Margin':value_list[4],
                                            'Operating Margin':value_list[5],
                                            'Return on Assets':value_list[6],
                                            'Return on Equity':value_list[7],
                                            'Revenue Per Share':value_list[8],
                                            'Market Cap':value_list[9],
                                             'Enterprise Value':value_list[10],
                                             'Forward P/E':value_list[11],
                                             'PEG Ratio':value_list[12],
                                             'Enterprise Value/Revenue':value_list[13],
                                             'Enterprise Value/EBITDA':value_list[14],
                                             'Revenue':value_list[15],
                                             'Gross Profit':value_list[16],
                                             'EBITDA':value_list[17],
                                             'Net Income Avl to Common ':value_list[18],
                                             'Diluted EPS':value_list[19],
                                             'Earnings Growth':value_list[20],
                                             'Revenue Growth':value_list[21],
                                             'Total Cash':value_list[22],
                                             'Total Cash Per Share':value_list[23],
                                             'Total Debt':value_list[24],
                                             'Current Ratio':value_list[25],
                                             'Book Value Per Share':value_list[26],
                                             'Cash Flow':value_list[27],
                                             'Beta':value_list[28],
                                             'Held by Insiders':value_list[29],
                                             'Held by Institutions':value_list[30],
                                             'Shares Short (as of':value_list[31],
                                             'Short Ratio':value_list[32],
                                             'Short % of Float':value_list[33],
                                             'Shares Short (prior ':value_list[34],
                                            'Status':status},
                                           ignore_index=True)
                    except Exception as e:
#                         print (e, 'here')
                        pass
#                     print (ticker, soup.find_all("td", {"class":"yfnc_tabledata1"})[26].text)
        except Exception as e:
             print( e, "NOT ------")
        
    #Graphing Data
#     for each_ticker in ticker_list:
#         try:
#             plot_df = df[(df['Ticker'] == each_ticker)]
#             plot_df = plot_df.set_index(['Date'])
#             if plot_df['Status'][-1] == "underperformed":
#                 color = 'r'
#             else:
#                 color = 'g'
#             plot_df['Difference'].plot(label =each_ticker, color= color)
#             plt.legend()
#             plt.ylabel("Difference of Stock % Change and SP500 % Change")
#             plt.title("S&P 500 Stocks Performance")
            
#         except:
#             print(e, "while graphing")
            
#     plt.show()
#     #save to CSV
#     name_to_save = gather.replace(" ", "").replace(')', "").replace('(', "").replace("/", "")+ str('.csv')
#     print(name_to_save)
    df.to_csv("key_stats.csv")
    
#  value = source.split(gather+':</td><td class="yfnc_tabledata1">')[1].split('</td>')[0]
   
    #print ([x[0] for x in os.walk(statspath)])
#     for each_dir in stock_list[1:]:
#         print(each_dir.split("/")[2])
    
Key_Stats()

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out o

# Using SVC

In [30]:
import numpy as np
from sklearn import svm, preprocessing
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import style
from sklearn.model_selection import train_test_split
style.use("ggplot")

In [33]:
FEATURES =  ['DE Ratio',
             'Trailing P/E',
             'Price/Sales',
             'Price/Book',
             'Profit Margin',
             'Operating Margin',
             'Return on Assets',
             'Return on Equity',
             'Revenue Per Share',
             'Market Cap',
             'Enterprise Value',
             'Forward P/E',
             'PEG Ratio',
             'Enterprise Value/Revenue',
             'Enterprise Value/EBITDA',
             'Revenue',
             'Gross Profit',
             'EBITDA',
             'Net Income Avl to Common ',
             'Diluted EPS',
             'Earnings Growth',
             'Revenue Growth',
             'Total Cash',
             'Total Cash Per Share',
             'Total Debt',
             'Current Ratio',
             'Book Value Per Share',
             'Cash Flow',
             'Beta',
             'Held by Insiders',
             'Held by Institutions',
             'Shares Short (as of',
             'Short Ratio',
             'Short % of Float',
             'Shares Short (prior ']
def Build_Data_Set():
    data_df = pd.DataFrame.from_csv("key_stats.csv")
    X = np.array(data_df[FEATURES].values.tolist())
    y = (data_df["Status"].replace("underperformed", 0).replace("outperformed market", 1).values.tolist())
    
    #Normalizing X
    X = preprocessing.scale(X)
    print(len(X))
    return X,y
def Analysis():
    
    test_size = 500
    X, y = Build_Data_Set()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    clf = svm.SVC(kernel = "linear", C = 1.0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    print("Accuracy", accuracy) #Accuracy on whether or not we predicted correctly if the stock would outperform or underperform the market
        
#     w = clf.coef_[0]
#     a = -w[0] /w[1]

Analysis()

/Users/sparajuli/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:160: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


3049
Accuracy 0.698360655738


# Scraping Recent Data

In [13]:
#!/usr/local/env python
import bs4 as bs
import urllib.request
import re
import json
import requests
import os
from threading import Thread


In [14]:
symbols = ['ABT', 'ABBV', 'ACN', 'ACE', 'ADBE', 'ADT', 'AAP', 'AES', 'AET', 'AFL', 'AMG', 'A', 'GAS', 'APD', 'ARG'
          ,'AKAM', 'AA', 'AGN', 'ALXN', 'ALLE', 'ADS', 'ALL', 'ALTR', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AMP', 'ABC',
          'AME', 'AMGN', 'APH', 'APC', 'ADI', 'AON', 'APA', 'AIV', 'AMAT', 'ADM', 'AIZ', 'T', 'ADSK', 'ADP', 'AN', 'AZO', 'AVGO', 'AVB', 'AVY',
          'BHI', 'BLL', 'BAC', 'BK', 'BCR', 'BXLT', 'BAX', 'BBT', 'BDX', 'BBBY', 'BRK-B', 'BBY', 'BLX', 'HRB', 'BA', 'BWA', 'BXP',
          'BSK', 'BMY', 'BRCM', 'BF-B', 'CHRW', 'CA', 'CVC', 'COG', 'CAM', 'CPB', 'COF', 'CAH', 'HSIC', 'KMX', 'CCL', 'CAT', 'CBG', 'CBS',
          'CELG', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHK', 'CVX', 'CMG', 'CB', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CTXS', 'CLX', 'CME', 'CMS', 'COH', 'KO', 'CCE', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CSC', 'CAG', 'COP', 'CNX', 'ED', 'STZ', 'GLW', 'COST', 'CCI', 
          'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DLPH', 'DAL', 'XRAY', 'DVN', 'DO', 'DTV', 'DFS', 'DISCA', 'DISCK', 'DG', 'DLTR', 'D', 'DOV', 'DOW', 'DPS', 'DTE', 'DD', 'DUK', 'DNB', 'ETFC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMC', 'EMR', 'ENDP', 'ESV', 'ETR', 'EOG', 'EQT', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ES', 'EXC', 'EXPE', 'EXPD', 'ESRX', 'XOM', 'FFIV', 'FB',
          'FAST', 'FDX', 'FIS', 'FITB', 'FSLR', 'FE', 'FSIV', 'FLIR', 'FLS', 'FLR', 'FMC', 'FTI', 'F', 'FOSL', 'BEN', 'FCX', 'FTR', 'GME', 'GPS', 'GRMN', 'GD', 'GE', 'GGP', 'GIS', 'GM', 'GPC', 'GNW', 'GILD', 'GS', 'GT', 'GOOGL', 'GOOG', 'GWW', 'HAL', 'HBI', 'HOG', 'HAR', 'HRS', 'HIG', 'HAS', 'HCA', 'HCP', 'HCN', 'HP', 'HES', 'HPQ', 'HD', 'HON', 'HRL', 'HSP', 'HST', 'HCBK', 'HUM', 'HBAN', 'ITW', 'IR', 'INTC', 'ICE', 'IBM', 'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IRM', 'JEC', 'JBHT', 'JNJ', 'JCI', 'JOY', 'JPM', 'JNPR',
         'KSU', 'K', 'KEY', 'GMCR', 'KMB', 'KIM', 'KMI', 'KLAC', 'KSS', 'KRFT', 'KR', 'LB', 'LLL', 'LH', 'LRCX', 'LM', 'LEG', 'LEN', 
          'LVLT', 'LUK', 'LLY', 'LNC', 'LLTC', 'LMT', 'L', 'LOW', 'LYB', 'MTB', 'MAC', 'M', 'MNK', 'MRO', 'MPC', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MAT', 'MKC', 'MCD', 'MHFI', 'MCK', 'MJN', 'MMV', 'MDT', 'MRK', 'MET', 'KORS', 'MCHP', 'MU', 'MSFT', 'MHK', 'TAP', 'MDLZ', 'MON', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MUR', 'MYL', 'NDAQ', 'NOV', 'NAVI', 'NTAP', 'NFLX', 'NWL', 'NFX', 'NEM', 'NWSA', 'NEE', 'NLSN', 'NKE', 'NI', 'NE', 'NBL', 'JWN', 'NSC', 'NTRS', 'NOC', 'NRG', 'NUE', 'NVDA', 'ORLY', 'OXY', 'OMC', 'OKE', 'ORCL', 'OI', 'PCAR', 'PLL', 'PH', 'PDCO', 'PAYX', 'PNR', 'PBCT', 'POM', 'PEP', 'PKI', 'PRGO', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PXD', 'PBI', 'PCL', 'PNC', 'RL', 'PPG', 'PPL', 'PX', 'PCP', 'PCLN', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RRC', 'RTN', 'O', 'RHT', 'REGN', 'RF', 'RSG', 'RAI', 'RHI', 'ROK', 'COL', 'ROP', 'ROST', 'RLC', 'R', 'CRM', 'SNDK', 'SCG', 'SLB', 'SNI', 'STX', 'SEE', 'SRE', 'SHW', 'SIAL', 'SPG', 'SWKS', 'SLG', 'SJM', 'SNA', 'SO', 'LUV', 'SWN', 'SE', 'STJ', 'SWK', 'SPLS', 'SBUX', 'HOT', 'STT', 'SRCL', 'SYK', 'STI', 'SYMC', 'SYY', 'TROW', 'TGT', 'TEL', 'TE', 'TGNA', 'THC', 'TDC', 'TSO', 'TXN', 'TXT', 'HSY', 'TRV', 'TMO', 'TIF', 'TWX', 'TWC', 'TJK', 'TMK', 'TSS', 'TSCO', 'RIG', 'TRIP', 'FOXA', 'TSN', 'TYC', 'UA', 'UNP', 'UNH', 'UPS', 'URI', 'UTX', 'UHS', 'UNM', 'URBN', 'VFC', 'VLO', 'VAR', 'VTR', 'VRSN', 'VZ', 'VRTX', 'VIAB', 'V', 'VNO', 'VMC', 'WMT', 'WBA', 'DIS', 'WM', 'WAT', 'ANTM', 'WFC', 'WDC', 'WU', 'WY', 'WHR', 'WFM', 'WMB', 'WEC', 'WYN', 'WYNN', 'XEL', 'XRX', 'XLNX', 'XL', 'XYL', 'YHOO', 'YUM', 'ZBH', 'ZION', 'ZTS']


# Daily closing prices to file

In [16]:
for symbol in symbols:
    script_dir = os.path.dirname('__file__')
    print (script_dir)
    rel_path = "daily_prices/"
    abs_file_path = os.path.join(script_dir, rel_path)
    print(symbol)
    myfile = open(abs_file_path + symbol + ".txt", "w+")
    myfile.close()
    url = "https://www.bloomberg.com/markets/api/bulk-time-series/price/"+symbol+"%3AUS?timeFrame=1_DAY"
    rawdata = urllib.request.urlopen(url).read()
    jsonData = json.loads(rawdata.decode('utf-8'))
    myfile = open(abs_file_path + symbol + ".txt", "a")
    for element in jsonData[0]['price']:
        myfile.write(str(symbol) + "," + str(element["dateTime"]) + "," + str(element["value"]) + "\n")
    myfile.close()


ABT

ABBV

ACN

ACE

ADBE

ADT

AAP

AES

AET

AFL

AMG

A

GAS

APD

ARG

AKAM

AA

AGN

ALXN


KeyboardInterrupt: 

In [17]:
def th(url):
    rawdata = urllib.request.urlopen(url).read()
    jsonData = json.loads(rawdata.decode('utf-8'))
#     myfile = open(abs_file_path + symbol + ".txt", "a")
    for element in jsonData[0]['price']:
        print(str(symbol) + "," + str(element["dateTime"]) + "," + str(element["value"]) + "\n")
    

In [ ]:
%%time
threadlist = []

for symbol in symbols:
    url = "https://www.bloomberg.com/markets/api/bulk-time-series/price/"+symbol+"%3AUS?timeFrame=1_DAY"
    t = Thread(target =th, args = (url,))
    t.start()
    threadlist.append(t)
    
for b in threadlist:
    b.join()


Exception in thread Thread-1591:
Traceback (most recent call last):
  File "/Users/sparajuli/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/sparajuli/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-17-03fe09de8d37>", line 2, in th
    rawdata = urllib.request.urlopen(url).read()
  File "/Users/sparajuli/anaconda3/lib/python3.6/urllib/request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/sparajuli/anaconda3/lib/python3.6/urllib/request.py", line 532, in open
    response = meth(req, response)
  File "/Users/sparajuli/anaconda3/lib/python3.6/urllib/request.py", line 642, in http_response
    'http', request, response, code, msg, hdrs)
  File "/Users/sparajuli/anaconda3/lib/python3.6/urllib/request.py", line 570, in error
    return self._call_chain(*args)
  File "/Users/sparajuli/anaconda3/lib/python3.6/urllib/request.py", 

In [16]:
%%time
print('hey')

hey
CPU times: user 470 µs, sys: 366 µs, total: 836 µs
Wall time: 555 µs
